In [1]:
import lxml.html
import lxml.etree
import requests
from datetime import datetime
from tqdm import tqdm, tqdm_notebook
from sqlalchemy import func, create_engine
from sqlalchemy import Column, Integer, Text, DateTime
from sqlalchemy.schema import Index
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base

In [2]:
Base = declarative_base()

class News(Base):
    __tablename__ = 'news'

    id = Column(Integer, primary_key=True)
    body = Column(Text, nullable=False)
    category = Column(Text, nullable=False)
    
    def __init__(self, body, category):
        self.body = body
        self.category = category


def init_db(db_url):
    engine = create_engine(db_url)
    Base.metadata.bind = engine
    Base.metadata.create_all()
    return sessionmaker(bind=engine)

db_session = init_db('sqlite:///lenta.db')()

In [3]:
def followTheLink(link):
    url = 'https://www.lenta.ru' + link
    response = requests.get(url)
    return lxml.html.fromstring(response.text)

In [4]:
def processItem(tree):
    category = tree.xpath('//a[@class="b-header-inner__block"]/text()')[0]
    body = ' '.join(tree.xpath('//div[@itemprop="articleBody"]/p/text()')).replace(u'\xa0', u' ')
    return category, body

In [5]:
now = datetime.now()
now_url = '/{}/{}/{}'.format(now.year, now.month, now.day)
tree = followTheLink(now_url) #начинаем с архива за сегодняшнее число

In [62]:
now_url = '/{}/{}/{}'.format('2014', '04', '15')
tree = followTheLink(now_url)

In [ ]:
date = []
day = 0
while '2012' not in date: #переходим на предыдущее число, пока не встретим 2014
    date = tree.xpath('//div[@class="b-archive__header-rubric"]/h1/span/text()')[0].split()
    day += 1
    print(day 
    item_urles = tree.xpath('//section[@class="b-longgrid-column"]/div/div[2]/h3/a/@href')
    
    for url in item_urles:
        itemTree = followTheLink(url)
        category, body = processItem(itemTree)
        news = News(body, category)
        db_session.add(news)
    db_session.commit()
    next_page_url = tree.xpath('//a[@class="control_mini"]/@href')[0]
    followTheLink(next_page_url)

In [52]:
#db_session.query(News).filter(News.category == '69-я параллель').delete()
#db_session.query(News).filter(News.category == 'Мир').delete()
db_session.query(News).filter(News.category == 'Бывший СССР', News.id > 146603).delete()
db_session.commit()

In [50]:
get_id = [instance.id for instance in db_session.query(News).filter(News.category == 'Бывший СССР').order_by(News.id)]

In [51]:
get_id[15000]
    

146603

In [5]:
cats = dict()
for instance in db_session.query(News).order_by(News.id):
    if instance.category in cats:
        cats[instance.category] += 1
    else:
        cats[instance.category] = 1
            
print(cats)

{'Наука и техника': 15001, 'Экономика': 15001, 'Россия': 15001, 'Культура': 10385, 'Спорт': 15001, 'Ценности': 8206, 'Из жизни': 14899, 'Интернет и СМИ': 15001, 'Силовые структуры': 15001, 'Бывший СССР': 15001}


In [6]:
categories = dict()
for i, cat in enumerate(db_session.query(News.category).distinct()):
    categories[cat[0]] = i
print(categories)

{'Наука и техника': 0, 'Экономика': 1, 'Россия': 2, 'Культура': 3, 'Спорт': 4, 'Ценности': 5, 'Из жизни': 6, 'Интернет и СМИ': 7, 'Силовые структуры': 8, 'Бывший СССР': 9}


In [10]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression 
from sklearn.pipeline import Pipeline
from sklearn import svm

#pipeline = Pipeline([
#    ('vect', CountVectorizer(max_df=0.2, ngram_range=(1,2))),
#    ('tfidf', TfidfTransformer()),
#    ('clf', SGDClassifier(penalty='elasticnet')),
#])
text_clf = Pipeline([('vectorizer', CountVectorizer(max_df=0.2, ngram_range=(1,2))),
                         ('tfidf', TfidfTransformer()),
                         ('clf', SGDClassifier(penalty='elasticnet'))])

text_clf = text_clf.fit([instance.body for instance in db_session.query(News).order_by(News.id)],
                       [categories[i.category] for i in db_session.query(News).order_by(News.id)])


#vectorizer = CountVectorizer()
#tfidf_transformer = TfidfTransformer()
#x_vect = vectorizer.fit_transform([instance.body for instance in db_session.query(News).order_by(News.id)])
#x_tfidf = tfidf_transformer.fit_transform(x_vect)
#num_cats = [categories[instance.category] for instance in db_session.query(News).order_by(News.id)]
#clf = SGDClassifier().fit(x_tfidf, num_cats)

In [19]:
data = ['нейтрон', 
        'Сенат США поддержал предложенный администрацией президента Дональда Трампа план налоговой реформы. Теперь два похожих, но немного отличающихся варианта законопроекта должны быть согласованы и приняты обеими палатами, после чего отправлены на подпись главе государства. Реформа является беспрецедентной по масштабам со времен Рональда Рейгана, но скептики считают, что она может все испортить. В частности, звучат опасения по поводу раздувания бюджетного дефицита, а также из-за роста и без того значительной разницы в доходах бедных и богатых.',  
        'Грабитель ворвался в магазин и, угрожая продавщице пистолетом, потребовал открыть кассу. В это время мимо торговой точки проходила женщина, и у преступника при виде свидетеля сдали нервы — он разбил витрину, схватил товар и выбежал на улицу.', 
        'Блогер из Британии, известный в сети под ником jamielliottg, поссорил два голосовых помощника Amazon Echo. Ролик опубликован на его YouTube-канале.', 
        'Картина «Первому игроку приготовиться» выйдет на экраны 28 марта 2018 года. Она основана на одноименном научно-фантастическом романе Эрнеста Клайна. По сюжету, в недалеком будущем мировая экономика оказалась в критическом положении из-за дефицита ресурсов, и люди находят спасение в иммерсивной онлайн-игре, в которую можно погружаться при помощи VR-шлема и костюма.', 
        'футбол',
        'Свадебный наряд состоит из топа с длинными рукавами и фестонами, застегивающегося на молнию по примеру спортивной куртки, и белых тренировочных штанов с широким передником с оборками из нескольких слоев полупрозрачного газа. Костюм дополняет фата. Создатель необычного наряда — креативный директор марки Vetements модельер Демна Гвасалия.']

print(text_clf.predict(data))

[0 1 2 4 3 4 1]


In [60]:
import tqdm
from time import time
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression 
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier()),
])
parameters = {
    'vect__max_df': (0.5, 0.75, 1.0),
    # 'vect__max_features': (None, 5000, 10000, 50000),
    #'vect__ngram_range': ((1, 1), (1, 2)),  # unigrams or bigrams
    #'tfidf__use_idf': (True, False),
    #'tfidf__norm': ('l1', 'l2'),
    #'clf__alpha': (0.00001, 0.000001),
    'clf__penalty': ('l2', 'elasticnet'),
    # 'clf__n_iter': (10, 50, 80),
}

grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1, verbose=1)

print("Performing grid search...")
print("pipeline:", [name for name, _ in pipeline.steps])
print("parameters:")
print(parameters)
t0 = time()
grid_search.fit([instance.body for instance in db_session.query(News).order_by(News.id)], 
                [categories[i.category] for i in db_session.query(News).order_by(News.id)])
print("done in %0.3fs" % (time() - t0))
print()

print("Best score: %0.3f" % grid_search.best_score_)
print("Best parameters set:")
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Performing grid search...
pipeline: ['vect', 'tfidf', 'clf']
parameters:
{'vect__max_df': (0.5, 0.75, 1.0), 'clf__penalty': ('l2', 'elasticnet')}
Fitting 3 folds for each of 6 candidates, totalling 18 fits


[Parallel(n_jobs=-1)]: Done  18 out of  18 | elapsed:  3.8min finished


done in 265.122s

Best score: 0.943
Best parameters set:
	clf__penalty: 'elasticnet'
	vect__max_df: 0.5
